In [25]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [26]:
# Try to avoid having to re-run this because it takes a while
crime_2020_2024 = '1V9pJ1fghA9FIYP1PgR_lfx-3lAMJPxWo'
crime_2010_2019 = '1HfsVnsIApb0wC05yQevKZTe_G71qmZ5O'

url_2010=f'https://drive.usercontent.google.com/download?id={crime_2010_2019}&export=download&authuser=0&confirm=t'
url_2020=f'https://drive.usercontent.google.com/download?id={crime_2020_2024}&export=download&authuser=0&confirm=t'

df_2020 = pd.read_csv(url_2020, on_bad_lines='skip')
df_2010 = pd.read_csv(url_2010, on_bad_lines='skip')


In [30]:
df_2010 = df_2010.rename(columns={'AREA ': 'AREA'})
df = pd.concat([df_2010, df_2020])
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,0,M,H,103.0,ALLEY,NaN,NaN,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,1100 0400 1402,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",0400,47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [31]:
cols_to_drop = ['Cross Street', 'Part 1-2', 'LOCATION', 'Rpt Dist No', 'DR_NO']
df = df.drop(columns=cols_to_drop)
df.columns

Index(['Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Crm Cd',
       'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent',
       'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status',
       'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LAT',
       'LON'],
      dtype='object')

In [32]:
df = df.rename(columns={
    'Date Rptd': 'reportDate',
    'DATE OCC': 'crimeDate',
    'TIME OCC': 'crimeTime',
    'AREA': 'area',
    'AREA NAME': 'areaName',
    'Crm Cd': 'crimeCode',
    'Crm Cd Desc': 'crimeCodeDescription',
    'Mocodes': 'moCodes',
    'Vict Age': 'victimAge',
    'Vict Sex': 'victimSex',
    'Vict Descent': 'victimDescent',
    'Premis Cd': 'siteCode',
    'Premis Desc': 'siteDescription',
    'Weapon Used Cd': 'weaponCode',
    'Weapon Desc': 'weaponDescription',
    'Status': 'status',
    'Status Desc': 'statusDescription',
    'Crm Cd 1': 'crimeCode1',
    'Crm Cd 2': 'crimeCode2',
    'Crm Cd 3': 'crimeCode3',
    'Crm Cd 4': 'crimeCode4',
    'LAT': 'lat',
    'LON': 'lon'
})

,reportDate,crimeDate,crimeTime,area,areaName,crimeCode,crimeCodeDescription,moCodes,victimAge,victimSex,victimDescent,siteCode,siteDescription,weaponCode,weaponDescription,status,statusDescription,crimeCode1,crimeCode2,crimeCode3,crimeCode4,lat,lon
0,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,AA,Adult Arrest,900.0,NaN,NaN,NaN,33.9825,-118.2695
1,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,33.9599,-118.3962
2,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,946,OTHER MISCELLANEOUS CRIME,0344,0,M,H,103.0,ALLEY,NaN,NaN,IC,Invest Cont,946.0,NaN,NaN,NaN,34.0224,-118.2524
3,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,900,VIOLATION OF COURT ORDER,1100 0400 1402,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,900.0,998.0,NaN,NaN,34.1016,-118.3295
4,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,122,"RAPE, ATTEMPTED",0400,47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,122.0,NaN,NaN,NaN,34.0387,-118.2488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970755,07/24/2024 12:00:00 AM,07/23/2024 12:00:00 AM,1400,7,Wilshire,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,34.0362,-118.3284
970756,01/15/2024 12:00:00 AM,01/15/2024 12:00:00 AM,100,1,Central,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 0400 0416,0,X,X,503.0,HOTEL,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,745.0,NaN,NaN,NaN,34.0685,-118.2460
970757,07/19/2024 12:00:00 AM,07/19/2024 12:00:00 AM,757,17,Devonshire,888,TRESPASSING,1501,0,X,X,885.0,MTA - ORANGE LINE - CHATSWORTH,NaN,NaN,IC,Invest Cont,888.0,NaN,NaN,NaN,34.2500,-118.5990
970758,04/24/2024 12:00:00 AM,04/24/2024 12:00:00 AM,1500,3,Southwest,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",1822 0334 0416 0445 0449 1202,70,F,W,102.0,SIDEWALK,308.0,STICK,IC,Invest Cont,230.0,NaN,NaN,NaN,34.0215,-118.2868


In [20]:
crime_categories = pd.DataFrame(df['crimeCodeDescription'].value_counts())
crime_categories.to_csv('crime_categories')

In [23]:
df['crimeCodeDescription'] = df['crimeCodeDescription'].str.lower()

theft_kws = ['theft', 'stolen', 'steal', 'snatching']

df['crimeTheft'] = df['crimeCodeDescription'].apply(lambda x: 1 if any([word in x for word in theft_kws]) else 0)

df

,reportDate,crimeDate,crimeTime,area,areaName,crimeCode,crimeCodeDescription,moCodes,victimAge,victimSex,victimDescent,siteCode,siteDescription,weaponCode,weaponDescription,status,statusDescription,crimeCode1,crimeCode2,crimeCode3,crimeCode4,lat,lon,crimeTheft
0,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,900,violation of court order,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,AA,Adult Arrest,900.0,NaN,NaN,NaN,33.9825,-118.2695,0
1,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,740,"vandalism - felony ($400 & over, all church va...",0329,0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,33.9599,-118.3962,0
2,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,946,other miscellaneous crime,0344,0,M,H,103.0,ALLEY,NaN,NaN,IC,Invest Cont,946.0,NaN,NaN,NaN,34.0224,-118.2524,0
3,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,900,violation of court order,1100 0400 1402,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,900.0,998.0,NaN,NaN,34.1016,-118.3295,0
4,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,122,"rape, attempted",0400,47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,122.0,NaN,NaN,NaN,34.0387,-118.2488,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970755,07/24/2024 12:00:00 AM,07/23/2024 12:00:00 AM,1400,7,Wilshire,510,vehicle - stolen,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,34.0362,-118.3284,1
970756,01/15/2024 12:00:00 AM,01/15/2024 12:00:00 AM,100,1,Central,745,vandalism - misdeameanor ($399 or under),0329 0400 0416,0,X,X,503.0,HOTEL,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,745.0,NaN,NaN,NaN,34.0685,-118.2460,0
970757,07/19/2024 12:00:00 AM,07/19/2024 12:00:00 AM,757,17,Devonshire,888,trespassing,1501,0,X,X,885.0,MTA - ORANGE LINE - CHATSWORTH,NaN,NaN,IC,Invest Cont,888.0,NaN,NaN,NaN,34.2500,-118.5990,0
970758,04/24/2024 12:00:00 AM,04/24/2024 12:00:00 AM,1500,3,Southwest,230,"assault with deadly weapon, aggravated assault",1822 0334 0416 0445 0449 1202,70,F,W,102.0,SIDEWALK,308.0,STICK,IC,Invest Cont,230.0,NaN,NaN,NaN,34.0215,-118.2868,0


In [22]:
df[df['crimeTheft']==1]

,reportDate,crimeDate,crimeTime,area,areaName,crimeCode,crimeCodeDescription,moCodes,victimAge,victimSex,victimDescent,siteCode,siteDescription,weaponCode,weaponDescription,status,statusDescription,crimeCode1,crimeCode2,crimeCode3,crimeCode4,lat,lon,crimeTheft


In [24]:
df['status'].value_counts()

status
IC    2400205
AO     361448
AA     305547
JA      19122
JO       7297
CC         35
TH          1
13          1
19          1
Name: count, dtype: int64